In [1]:
import preprocessing as pp
import sampling
import random
import torch
from models import simpleGCN
from torch_geometric.loader import DataLoader
from tqdm import tqdm
import numpy as np
import pandas as pd

In [2]:

nodesFileNerve =  "~/Documents/Intestine/nerve-mask/nodes_nerve_bs2.csv"
edgesFileNerve = "~/Documents/Intestine/nerve-mask/edges_nerve_bs2.csv"

nodesFileLymph =  "~/Documents/Intestine/lymph-mask/nodes_lymph_bs2.csv"
edgesFileLymph = "~/Documents/Intestine/lymph-mask/edges_lymph_bs2.csv"





nodes_n = pd.read_csv(nodesFileNerve, sep = ";", index_col= "id")
edges_n = pd.read_csv(edgesFileNerve, sep = ";", index_col= "id")
nodes_l = pd.read_csv(nodesFileLymph, sep = ";", index_col= "id")
edges_l = pd.read_csv(edgesFileLymph, sep = ";", index_col= "id")

#Probier mal 1,625 für x,y und 6 für z
nodes_n["pos_x"] = nodes_n["pos_x"]*1.65
nodes_n["pos_y"] = nodes_n["pos_y"]*1.65
nodes_n["pos_z"] = nodes_n["pos_z"]*6

nodes_l["pos_x"] = nodes_l["pos_x"]*1.65
nodes_l["pos_y"] = nodes_l["pos_y"]*1.65
nodes_l["pos_z"] = nodes_l["pos_z"]*6


# create the graphs for both networks
G_nerve = pp.createGraph(nodes_n, edges_n, index_addon ="n")
G_lymph = pp.createGraph(nodes_l, edges_l, index_addon ="l")

# get short description of graph
pp.graphSummary(G_nerve)
pp.graphSummary(G_lymph)

# get rid of self-loops, multi edges and isolated nodes
G_nerve_einf = pp.convertToEinfach(G_nerve)
G_lymph_einf = pp.convertToEinfach(G_lymph)

# enrich the attributes of the nodes with information from the incident edges
pp.enrichNodeAttributes(G_lymph_einf)
pp.enrichNodeAttributes(G_nerve_einf)


***************
Number of Nodes: 4317
Number of Edges: 6634
Number of Connected Components: 22
Number of Self Loops: 528
Number of Isolated Nodes: 0
Average Node Degree: 3.073430623117906
***************
***************
Number of Nodes: 3036
Number of Edges: 3864
Number of Connected Components: 89
Number of Self Loops: 25
Number of Isolated Nodes: 1
Average Node Degree: 2.5454545454545454
***************


# Graph Classification using Subsets of Lymph and Nerve Network for Training and Testing

In [3]:

# create random samples 
randomSampleLymphNx, randomSampleLymph = sampling.randomGeomSubgraphs(G_lymph_einf, label = 1,starts = 100, node_sample_size = 100,  mode = "rnn")
randomSampleNerveNx, randomSampleNerve = sampling.randomGeomSubgraphs(G_nerve_einf, label = 0,starts = 100, node_sample_size = 100,  mode = "rnn")

# combine the graphs to a random set
allGraphs = randomSampleLymph + randomSampleNerve
random.shuffle(allGraphs)

# split into training and test set
breaker = int(len(allGraphs)*0.8)
train_dataset = allGraphs[:breaker]
test_dataset = allGraphs[breaker:]

Creating subgraphs using random node neighbor selection.: 100%|██████████| 100/100 [00:00<00:00, 417.40it/s]
Creating subgraphs using random node neighbor selection.: 100%|██████████| 100/100 [00:00<00:00, 199.68it/s]


In [4]:
# selection of the features to use
slice = [1,3,7,8]

# create the model
model = simpleGCN.GCN(hidden_channels=32, in_features = len(slice))
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# create brach data loaders for training and test set
train_loader = DataLoader(train_dataset, batch_size= 64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)



def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x[:,slice], data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x[:,slice], data.edge_index, data.batch)  
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.



for epoch in range(1, 11):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')



# good features: 1,3 - both features that have to do with length                performance is best if just these 2 features are used
# good features: 7,8 - both features that have to do with distance
# medium features: 6 - num Voxels
# medium features: 4 - curveness
# trash feature: 0,2 - both features that have to do with minRadiusAvg
# trash feature: 5 - avgRadiusStd



# 16 hidden layers seems favorable compared to 64 or 8

# slice = [1,3,7,8] leads to very good results
# slice = [1,3] results are equal to 1,3,7,8 ... maybe better

Epoch: 001, Train Acc: 0.5232, Test Acc: 0.4737
Epoch: 002, Train Acc: 0.8146, Test Acc: 0.8421
Epoch: 003, Train Acc: 0.7881, Test Acc: 0.8421
Epoch: 004, Train Acc: 0.9073, Test Acc: 0.8947
Epoch: 005, Train Acc: 0.8808, Test Acc: 0.8947
Epoch: 006, Train Acc: 0.9073, Test Acc: 0.8947
Epoch: 007, Train Acc: 0.9007, Test Acc: 0.8947
Epoch: 008, Train Acc: 0.9007, Test Acc: 0.8947
Epoch: 009, Train Acc: 0.8940, Test Acc: 0.8947
Epoch: 010, Train Acc: 0.9073, Test Acc: 0.8947


# Node Classification using a contracted graph by spatial proximity to differentiate 3 node types (nerve, lymph, connected)

In [5]:
import networkx as nx 
import pandas as pd


In [6]:
#rename the nodes to have unqiue identifiers for different types
idx = list(nodes_n.index)
idx_new = [str(elem) + "n" for elem in idx]
nodes_n.index = idx_new

idx = list(nodes_l.index)
idx_new = [str(elem) + "l" for elem in idx]
nodes_l.index = idx_new

edges_n['node1id'] = edges_n['node1id'].apply(lambda x: str(x) + "n")
edges_n['node2id'] = edges_n['node2id'].apply(lambda x: str(x) + "n") 

edges_l['node1id'] = edges_l['node1id'].apply(lambda x: str(x) + "l")
edges_l['node2id'] = edges_l['node2id'].apply(lambda x: str(x) + "l") 



In [7]:
from scipy.spatial import KDTree
from scipy.sparse import csr_matrix
from scipy.sparse import dok_array
from scipy.sparse.csgraph import connected_components

sparse = pp.network_sparse_distance_matrix(nodes_n, nodes_l, th = 0.005)

print(len(sparse))

adjM = dok_array((nodes_n.shape[0]+ nodes_l.shape[0], nodes_n.shape[0]+ nodes_l.shape[0]), dtype=np.uint8)


for key in sparse.keys():
    adjM[key[0], key[1] + nodes_n.shape[0]] = 1
    adjM[key[1]+ nodes_n.shape[0], key[0]] = 1

adjMcsr = adjM.tocsr()




num, labels = connected_components(csgraph=adjMcsr, directed = False)


con_comp = {}
for i in range(len(labels)):
    try:
        con_comp[labels[i]].add(i)
    except KeyError:
        con_comp[labels[i]] = {i}

rel_comp = {}
for k, v in con_comp.items():
    if len(v)>1:
        resList = []
        for val in v:
            if val >=nodes_n.shape[0]:
                resList.append(str(val- nodes_n.shape[0]) + "l")
            else:
                resList.append(str(val) + "n")
        rel_comp[str(k) +"c"] = resList
sparse.keys()


reverse_dict= {}
for k, v in rel_comp.items():
    for val in v:
        reverse_dict[val] = k

1


In [9]:
merged_nodes = pd.concat([nodes_l.loc[:,["pos_x", "pos_y", "pos_z"]], nodes_n.loc[:,["pos_x", "pos_y", "pos_z"]]])
new_nodes = pd.DataFrame(columns=merged_nodes.columns )


# replace the contracted node with new nodes
# the position of the new node is an average of all the previous nodes
for k, valList in rel_comp.items():
    new_nodes.loc[k] = merged_nodes.loc[valList].mean()
    merged_nodes.drop(valList, inplace = True)

# concat the all nodes and the new nodes
merged_nodes = pd.concat([merged_nodes, new_nodes])

# createa a combined edge file
merged_edges = pd.concat([edges_l, edges_n])

# change the names of the edges to the new names
for idxE, edge in merged_edges.iterrows():
    if edge["node1id"] in reverse_dict.keys():
        merged_edges.loc[idxE,"node1id"] = reverse_dict[edge["node1id"]]
    if edge["node2id"] in reverse_dict.keys():
        merged_edges.loc[idxE,"node2id"] = reverse_dict[edge["node2id"]]


# create a new graph based on the old information

G_contract = pp.createGraph(merged_nodes, merged_edges)
G_contract_einf = pp.convertToEinfach(G_contract, self_loops = True)
pp.enrichNodeAttributes(G_contract_einf)





In [10]:
# create the ground truth for the node class
all_nodes = list(G_contract_einf.nodes)
nerve_class = np.array([elem[-1] == "n" for elem in all_nodes])*0
lymph_class = np.array([elem[-1] == "l" for elem in all_nodes])*1
combined_class = np.array([elem[-1] == "c" for elem in all_nodes])*2

class_assign = nerve_class+lymph_class+combined_class


# create the training and testing masks
train_mask = np.random.choice(np.arange(0, len(class_assign)), size=int(len(class_assign)*0.8), replace = False)
test_mask = np.arange(0, len(class_assign))[-train_mask]

train_mask= torch.tensor(train_mask)
test_mask= torch.tensor(test_mask)


# convert the graph to a networkx graph

from torch_geometric.utils.convert import from_networkx
networkXG = from_networkx(G_contract_einf)
networkXG.y = torch.tensor(class_assign)


In [11]:
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

num_feat = networkXG.x.shape[1]
num_class = len(np.unique(networkXG.y))

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(1234567)
        self.conv1 = GCNConv(num_feat, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_class)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

model = GCN(hidden_channels=16)
print(model)

GCN(
  (conv1): GCNConv(9, 16)
  (conv2): GCNConv(16, 3)
)


In [12]:
model = GCN(hidden_channels=16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = torch.nn.CrossEntropyLoss()

def train():
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(networkXG.x, networkXG.edge_index)  # Perform a single forward pass.
      loss = criterion(out[train_mask], networkXG.y[train_mask])  # Compute the loss solely based on the training nodes.
      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test():
      model.eval()
      out = model(networkXG.x, networkXG.edge_index)
      pred = out.argmax(dim=1)  # Use the class with highest probability.
      test_correct = pred[test_mask] == networkXG.y[test_mask]  # Check against ground-truth labels.
      test_acc = int(test_correct.sum()) / len(test_mask)  # Derive ratio of correct predictions.
      return test_acc


for epoch in range(1, 201):
    loss = train()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 001, Loss: 5.2234
Epoch: 002, Loss: 4.4516
Epoch: 003, Loss: 3.8277
Epoch: 004, Loss: 3.4694
Epoch: 005, Loss: 3.2425
Epoch: 006, Loss: 3.2330
Epoch: 007, Loss: 2.8034
Epoch: 008, Loss: 2.7739
Epoch: 009, Loss: 2.7145
Epoch: 010, Loss: 2.4212
Epoch: 011, Loss: 2.2999
Epoch: 012, Loss: 2.2584
Epoch: 013, Loss: 2.0246
Epoch: 014, Loss: 1.8189
Epoch: 015, Loss: 1.8078
Epoch: 016, Loss: 1.5616
Epoch: 017, Loss: 1.5491
Epoch: 018, Loss: 1.3697
Epoch: 019, Loss: 1.3723
Epoch: 020, Loss: 1.2982
Epoch: 021, Loss: 1.1069
Epoch: 022, Loss: 1.0839
Epoch: 023, Loss: 1.0152
Epoch: 024, Loss: 0.9448
Epoch: 025, Loss: 0.9017
Epoch: 026, Loss: 0.8620
Epoch: 027, Loss: 0.7750
Epoch: 028, Loss: 0.7381
Epoch: 029, Loss: 0.7288
Epoch: 030, Loss: 0.7235
Epoch: 031, Loss: 0.6710
Epoch: 032, Loss: 0.6380
Epoch: 033, Loss: 0.6377
Epoch: 034, Loss: 0.6005
Epoch: 035, Loss: 0.5915
Epoch: 036, Loss: 0.5875
Epoch: 037, Loss: 0.5781
Epoch: 038, Loss: 0.5668
Epoch: 039, Loss: 0.5625
Epoch: 040, Loss: 0.5664


In [ ]:
test_acc = test()
print(f'Test Accuracy: {test_acc:.4f}')

Test Accuracy: 0.7764
